Dataset : https://drive.google.com/drive/folders/1ZDVv9fsCaD3FdCp8RaoKKGFPfvmWbayd?usp=drive_link



This code implements a few-shot learning model for wheat disease classification using EfficientNet for feature extraction and an attention module for similarity learning, trained with cross-entropy loss and evaluated for accuracy.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np



In [2]:
# Define Dataset Class
class WheatDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [3]:
# Data Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Dataset and Split into Train/Test
def load_data(data_dir, train_split=0.8):
    """
    Load dataset and split into train and test.
    :param data_dir: Path to dataset where folders represent classes.
    :param train_split: Ratio of training data (default: 80% train, 20% test).
    :return: Train and test datasets.
    """
    classes = sorted(os.listdir(data_dir))  # Get class names
    image_paths = []
    labels = []
    class_to_idx = {cls: idx for idx, cls in enumerate(classes)}

    for cls in classes:
        class_path = os.path.join(data_dir, cls)
        if not os.path.isdir(class_path):
            continue  # Skip non-directory files
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            image_paths.append(img_path)
            labels.append(class_to_idx[cls])

    # Shuffle the dataset
    dataset = list(zip(image_paths, labels))
    random.shuffle(dataset)
    image_paths, labels = zip(*dataset)

    # Split into train and test
    train_size = int(train_split * len(image_paths))
    train_images, test_images = image_paths[:train_size], image_paths[train_size:]
    train_labels, test_labels = labels[:train_size], labels[train_size:]

    return train_images, train_labels, test_images, test_labels, class_to_idx


In [4]:
# Load Data
dataset_path = "./D3_Final"  # Update this with your dataset path
train_images, train_labels, test_images, test_labels, class_to_idx = load_data(dataset_path)

train_dataset = WheatDataset(train_images, train_labels, transform=transform)
test_dataset = WheatDataset(test_images, test_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [5]:
# Feature Extractor (EfficientNet)
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.efficient_net = models.efficientnet_b0(pretrained=True)
        self.efficient_net.classifier = nn.Identity()  # Remove last layer

    def forward(self, x):
        return self.efficient_net(x)



In [10]:
class AttentionModule(nn.Module):
    def __init__(self, feature_dim):
        super(AttentionModule, self).__init__()
        self.W1 = nn.Linear(feature_dim, feature_dim)
        self.W2 = nn.Linear(feature_dim, feature_dim)
        self.omega_x = nn.Parameter(torch.rand(feature_dim))
        self.omega_y = nn.Parameter(torch.rand(feature_dim))
        self.omega_xy = nn.Parameter(torch.rand(feature_dim))

    def forward(self, support, query):
        s = torch.relu(self.W1(support))
        q = torch.relu(self.W2(query))
        similarity = torch.sum(self.omega_x * s + self.omega_y * q + self.omega_xy * (s * q), dim=-1)

        # Ensure output matches EfficientNet feature size (1280)
        output = torch.relu(self.W1(support) + self.W2(query))
        return output  # Output must be (batch_size, 1280)


In [11]:
# Few-Shot Learning Network
class FewShotNetwork(nn.Module):
    def __init__(self, feature_dim=1280, num_classes=18):
        super(FewShotNetwork, self).__init__()
        self.feature_extractor = FeatureExtractor()
        self.attention = AttentionModule(feature_dim)
        self.fc = nn.Linear(feature_dim, num_classes)

    def forward(self, support, query):
        support_features = self.feature_extractor(support)
        query_features = self.feature_extractor(query)
        similarity = self.attention(support_features, query_features)
        output = self.fc(similarity)
        return output



In [12]:
# Model Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FewShotNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)



In [13]:
# Training Loop
def train_model(model, train_loader, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images, images)  # Support and Query Set (same for now)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}")

train_model(model, train_loader)


Epoch [1/20], Loss: 0.4844
Epoch [2/20], Loss: 0.2233
Epoch [3/20], Loss: 0.2047
Epoch [4/20], Loss: 0.1397
Epoch [5/20], Loss: 0.1512
Epoch [6/20], Loss: 0.1362
Epoch [7/20], Loss: 0.1829
Epoch [8/20], Loss: 0.0775
Epoch [9/20], Loss: 0.0683
Epoch [10/20], Loss: 0.0775
Epoch [11/20], Loss: 0.0542
Epoch [12/20], Loss: 0.0539
Epoch [13/20], Loss: 0.1432
Epoch [14/20], Loss: 0.0579
Epoch [15/20], Loss: 0.0291
Epoch [16/20], Loss: 0.0388
Epoch [17/20], Loss: 0.0397
Epoch [18/20], Loss: 0.0276
Epoch [19/20], Loss: 0.0629
Epoch [20/20], Loss: 0.0595


In [14]:
# Model Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images, images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

evaluate_model(model, test_loader)


Test Accuracy: 98.61%


--------------------- Add more visualisation --------------

In [ ]:
def evaluate_model(model, test_loader, class_names):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images, images)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calcul de la précision
    accuracy = np.mean(np.array(all_preds) == np.array(all_labels)) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

    # Génération de la Matrice de Confusion
    cm = confusion_matrix(all_labels, all_preds)
    
    # Affichage de la Matrice de Confusion
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Prédictions")
    plt.ylabel("Vraies Classes")
    plt.title("Matrice de Confusion")
    plt.show()

    # Rapport de Classification
    print("\nRapport de Classification :\n", classification_report(all_labels, all_preds, target_names=class_names))


In [ ]:
def plot_training_curve(train_losses, test_losses, test_accuracies):
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(12, 5))

    # Courbe de la Perte
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Train Loss", marker='o')
    plt.plot(epochs, test_losses, label="Test Loss", marker='o')
    plt.xlabel("Épochs")
    plt.ylabel("Perte")
    plt.title("Évolution de la Perte")
    plt.legend()

    # Courbe de la Précision
    plt.subplot(1, 2, 2)
    plt.plot(epochs, test_accuracies, label="Test Accuracy", marker='o', color='green')
    plt.xlabel("Épochs")
    plt.ylabel("Précision (%)")
    plt.title("Évolution de la Précision")
    plt.legend()

    plt.show()


In [ ]:
class_names = ["Brown rust", "Yellow rust", "Stem rust"]
evaluate_model(model, test_loader, class_names)
